<a href="https://colab.research.google.com/github/ricardogr07/Machine-Learning-Basics/blob/main/src/2_Feature_selection_backward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Your First Machine Learning Model: A Complete Guide to Backward Stepwise Feature Selection

In [ ]:
# Install the necessary libraries
%pip install numpy pandas scikit-learn statsmodels faraway

Libraries

In [ ]:
# Import essential libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Import dataset
import faraway.datasets.ozone as ozone

Base model

In [ ]:
# Load the Ozone dataset
data = ozone.load()

# Separate features (X) and the target variable (y)
X = data.drop(columns=['O3'])
y = data['O3']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Add constant term for intercept
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit OLS model
base_model = sm.OLS(y_train, X_train_const).fit()

# Predict on test set
y_pred = base_model.predict(X_test_const)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Performance
mse = mean_squared_error(y_test, y_pred)

# Create DataFrame to store the results
results_base_model = pd.DataFrame({
    "R-squared": [base_model.rsquared],
    "Adjusted R-squared": [base_model.rsquared_adj],
    "MSE": [mse]
}, index=["Base Model"])

display(results_base_model)

Forward stepwise

In [ ]:
def forward_stepwise_selection(X_train:pd.DataFrame, y_train:pd.DataFrame, initial_sig_vars:list):
    """
    Perform forward stepwise selection to maximize Adjusted R-squared.

    Parameters:
    X_train (pd.DataFrame): The training data with features.
    y_train (pd.Series or pd.DataFrame): The target variable.
    initial_sig_vars (list): A list of initial significant features to start with.

    Returns:
    tuple: Best Adjusted R-squared, best model, and the list of selected variables.
    """

    # Define a function to calculate Adjusted R-squared for a given set of features
    def calculate_adjusted_r2(X, y):
        model_OLS = sm.OLS(y, X).fit()
        return model_OLS.rsquared_adj, model_OLS

    # Select only the significant features for the initial model
    if len(initial_sig_vars) != 0 :
        X_train_significant_features = X_train[initial_sig_vars]
    else:
        X_train_significant_features = X_train

    # Initialize with significant features
    selected_vars = list(X_train_significant_features.columns)
    remaining_vars = [var for var in X_train.columns if var not in selected_vars and var != 'const']

    # Best model using initial significant features
    best_adj_r2, best_model = calculate_adjusted_r2(X_train_significant_features, y_train)

    # Forward Stepwise process: Add variables one at a time
    while remaining_vars:
        adj_r2_with_candidates = []
        for candidate in remaining_vars:
            # Add candidate variable to the current model
            X_candidate = X_train_significant_features.join(X_train[candidate])
            adj_r2, model = calculate_adjusted_r2(X_candidate, y_train)
            adj_r2_with_candidates.append((adj_r2, candidate, model))

        # Sort and select the best candidate variable (higher Adjusted R-squared is better)
        adj_r2_with_candidates.sort(reverse=True, key=lambda x: x[0])  # Sort in descending order to get the highest value
        best_new_adj_r2, best_new_var, best_new_model = adj_r2_with_candidates[0]

        # Update model if new Adjusted R-squared is higher
        if best_new_adj_r2 > best_adj_r2:
            selected_vars.append(best_new_var)
            X_train_significant_features = X_train_significant_features.join(X_train[best_new_var])
            best_adj_r2 = best_new_adj_r2
            best_model = best_new_model
            remaining_vars.remove(best_new_var)
        else:
            break

    return best_adj_r2, best_model, selected_vars

In [ ]:
# Add constant term for intercept
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

significant_features = ['humidity', 'temp', 'ibt', 'doy']

best_adj_r2, best_model, selected_vars = forward_stepwise_selection(X_train_const, y_train, significant_features)
print("Adjusted R-squared of Forward Stepwise Model:", best_adj_r2)
print("Selected Variables:", selected_vars)

# Predict on test set using the selected features
X_test_selected = X_test_const[selected_vars]
y_pred = best_model.predict(X_test_selected)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Create DataFrame to store the results
results_forward = pd.DataFrame({
    "R-squared": [best_model.rsquared],
    "Adjusted R-squared": [best_model.rsquared_adj],
    "MSE": [mse]
}, index=["Forward Stepwise Model"])

display(results_forward)

Alternative way using the `SequentialFeatureSelector` class

In [ ]:
# Forward Stepwise Feature Selection
model = LinearRegression()
forward_selector = SequentialFeatureSelector(model, direction='forward', scoring='r2')
forward_selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[forward_selector.get_support()]
print(f'Selected features: {selected_features}')

# Fit the model with selected features
X_train_selected = X_train[selected_features]
model.fit(X_train_selected, y_train)

# Predict using the selected features
y_train_pred = model.predict(X_train_selected)

# Calculate metrics
r_squared = r2_score(y_train, y_train_pred)
n = len(y_train)
p = X_train_selected.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
mse = mean_squared_error(y_train, y_train_pred)

# Create a DataFrame to display results
data = {
    'R-squared': [r_squared],
    'Adjusted R-squared': [adjusted_r_squared],
    'MSE': [mse]
}
df = pd.DataFrame(data, index=['Forward Stepwise Model'])

# Display the DataFrame
display(df)

Backward stepwise

In [ ]:
# Define a function to calculate Adjusted R-squared using OLS from statsmodels
def calculate_adjusted_r2(X, y):
    model_OLS = sm.OLS(y, X).fit()
    return model_OLS.rsquared_adj, model_OLS

# Backward Stepwise Selection function
def backward_stepwise_selection(X_train, y_train):
    """
    Perform backward stepwise selection to maximize Adjusted R-squared using OLS.

    Parameters:
    X_train (pd.DataFrame): The training data with features.
    y_train (pd.Series or pd.DataFrame): The target variable.

    Returns:
    tuple: Best Adjusted R-squared, best model, and the list of selected variables.
    """

    # Initialize the set of all variables
    selected_vars = list(X_train.columns)

    # Best model using all features
    best_adj_r2, best_model = calculate_adjusted_r2(X_train[selected_vars], y_train)

    # Backward Stepwise process: Remove variables one at a time
    while len(selected_vars) > 1:
        adj_r2_with_candidates = []

        # Try removing each variable one by one
        for candidate in selected_vars:
            remaining_vars = [var for var in selected_vars if var != candidate]
            adj_r2, model = calculate_adjusted_r2(X_train[remaining_vars], y_train)
            adj_r2_with_candidates.append((adj_r2, candidate, model))

        # Sort and select the best candidate to remove (highest Adjusted R-squared)
        adj_r2_with_candidates.sort(reverse=True, key=lambda x: x[0])  # Sort in descending order
        best_new_adj_r2, worst_var, best_new_model = adj_r2_with_candidates[0]

        # If removing a variable improves adjusted R-squared, update the model
        if best_new_adj_r2 > best_adj_r2:
            selected_vars.remove(worst_var)
            best_adj_r2 = best_new_adj_r2
            best_model = best_new_model
        else:
            break

    return best_adj_r2, best_model, selected_vars


In [ ]:
# Example usage
best_adj_r2, best_model, selected_features = backward_stepwise_selection(X_train_const, y_train)

# Output the results
print("Selected Features:", selected_features)

# Predict on test set using the selected features
X_test_selected = X_test_const[selected_features]
y_pred = best_model.predict(X_test_selected)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Create DataFrame to store the results
results_backward = pd.DataFrame({
    "R-squared": [best_model.rsquared],
    "Adjusted R-squared": [best_model.rsquared_adj],
    "MSE": [mse]
}, index=["Backward Stepwise Model"])

display(results_backward)

In [ ]:
# Backward Stepwise Feature Selection
model = LinearRegression()
forward_selector = SequentialFeatureSelector(model, direction='backward', scoring='r2')
forward_selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[forward_selector.get_support()]
print(f'Selected features: {selected_features}')

# Fit the model with selected features
X_train_selected = X_train[selected_features]
model.fit(X_train_selected, y_train)

# Predict using the selected features
y_train_pred = model.predict(X_train_selected)

# Calculate metrics
r_squared = r2_score(y_train, y_train_pred)
n = len(y_train)
p = X_train_selected.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
mse = mean_squared_error(y_train, y_train_pred)

# Create a DataFrame to display results
data = {
    'R-squared': [r_squared],
    'Adjusted R-squared': [adjusted_r_squared],
    'MSE': [mse]
}
df = pd.DataFrame(data, index=['Forward Stepwise Model'])

# Display the DataFrame
display(df)

In [ ]:
df = pd.concat([results_base_model, results_forward, results_backward])

display(df)

Second case

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import faraway.datasets.prostate as prostate

In [ ]:
prostate_data = prostate.load()

X_train_prostate = prostate_data.drop(columns=['svi'])
y_train_prostate = prostate_data['svi']

X_train_prostate, X_test_prostate, y_train_prostate, y_test_prostate = train_test_split(X_train_prostate, y_train_prostate, test_size=0.2, random_state=123)

In [ ]:
# Ajustar el modelo de regresión logística sin intercepto
model_LR_no_intercept = LogisticRegression(max_iter=1000)
model_LR_no_intercept.fit(X_train_prostate, y_train_prostate)

# Realizar predicciones en el conjunto de test
y_pred_prostate = model_LR_no_intercept.predict(X_test_prostate)

In [ ]:
# Calculate the classification accuracy
accuracy = accuracy_score(y_test_prostate, y_pred_prostate)

# Calculate additional metrics
precision = precision_score(y_test_prostate, y_pred_prostate)
recall = recall_score(y_test_prostate, y_pred_prostate)
f1 = f1_score(y_test_prostate, y_pred_prostate)

# For AUC-ROC, we need the prediction probabilities
y_pred_proba_prostate = model_LR_no_intercept.predict_proba(X_test_prostate)[:, 1]
auc_roc = roc_auc_score(y_test_prostate, y_pred_proba_prostate)

# Create a DataFrame with the metrics
metrics_data = {
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1': [f1],
    'AUC-ROC': [auc_roc]
}

# Create the DataFrame
base_metrics_df = pd.DataFrame(metrics_data, index=['Base Model'])

# Display the metrics DataFrame
display(base_metrics_df)

In [ ]:
# Calcular el VIF para cada columna
VIF_vars = []

for i in range(X_train_prostate.shape[1]):
    vif = variance_inflation_factor(X_train_prostate.values, i)
    VIF_vars.append(vif)

print("VIF_vars =", VIF_vars)

In [ ]:
# Crear DataFrame con los VIF
vif_data = pd.DataFrame({
    'Variable': ['lcavol', 'lweight', 'age', 'lbph', 'lcp', 'gleason', 'pgg45', 'lpsa'],
    'VIF': VIF_vars
})
variables_low_vif = vif_data[vif_data['VIF'] <= 10]['Variable'].tolist()
print("Variables con VIF <= 10:", variables_low_vif)

# Agregar una columna de constantes
X_train_prostate = sm.add_constant(X_train_prostate)
X_test_prostate = sm.add_constant(X_test_prostate)

# Inicializar el modelo con todas las variables
model = sm.Logit(y_train_prostate, X_train_prostate).fit()

# Aplicar backward stepwise
while True:
    # Obtener el valor-p para cada variable
    p_values = model.pvalues
    # Identificar la variable con el valor-p más alto
    max_p_value = p_values.max()
    if max_p_value < 0.05:
        break
    # Eliminar la variable menos significativa
    remove_var = p_values.idxmax()
    X_train_prostate = X_train_prostate.drop(columns=[remove_var])
    X_test_prostate = X_test_prostate.drop(columns=[remove_var])
    # Ajustar el modelo con las variables restantes
    model = sm.Logit(y_train_prostate, X_train_prostate).fit()

# Realizar predicciones en el conjunto de prueba
y_pred_prostate = model.predict(X_test_prostate) > 0.5
porcentaje_clasificacion_backward = accuracy_score(y_test_prostate, y_pred_prostate)

In [ ]:
print(X_train_prostate.columns)

# Calculate the classification accuracy
accuracy = accuracy_score(y_test_prostate, y_pred_prostate)

# Calculate additional metrics
precision = precision_score(y_test_prostate, y_pred_prostate)
recall = recall_score(y_test_prostate, y_pred_prostate)
f1 = f1_score(y_test_prostate, y_pred_prostate)

# For AUC-ROC, we need the prediction probabilities
y_pred_proba_prostate = model.predict(X_test_prostate)
auc_roc = roc_auc_score(y_test_prostate, y_pred_proba_prostate)

# Create a DataFrame with the metrics
metrics_data = {
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1': [f1],
    'AUC-ROC': [auc_roc]
}

# Create the DataFrame
backward_metrics_df = pd.DataFrame(metrics_data, index=['Backward Selection Model'])

display(backward_metrics_df)

In [ ]:
df = pd.concat([base_metrics_df, backward_metrics_df])

display(df)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test_prostate, y_pred_prostate)

# Plot the confusion matrix using Seaborn heatmap for better visualization
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Logistic Regression")
plt.show()
